# Comparaison des Réponses AI : <br>Prompts Simples vs. Elaborés

### Ce notebook démontre comment générer et comparer <br>les réponses d'un modèle AI en utilisant des prompts <br>simples et élaborés.

# Imports

Nous importons les bibliothèques nécessaires :

In [1]:
import pandas as pd
import re
import google.generativeai as genai

# Configuration de la clé API pour Gemini

In [ ]:
genai.configure(api_key='YOUR_API_KEY')
model = genai.GenerativeModel('gemini-pro')

# Prompt utilisateur simple

In [3]:
# Entrée utilisateur
user_request = "Je voudrais apprendre à nager. Donnez-moi des recommandations pour ce hobby avec une analyse détaillée."
simple_response = model.generate_content(user_request).text      

In [4]:
print(simple_response)

**Recommandations pour apprendre à nager**

**Niveau débutant**

* **Trouvez un moniteur certifié :** Un moniteur qualifié peut vous enseigner les techniques appropriées et vous fournir un soutien personnalisé.
* **Commencez par l'eau peu profonde :** Commencez dans un bassin ou une zone d'eau où vous pouvez vous tenir debout. Cela vous donnera confiance et vous permettra de pratiquer les mouvements de base.
* **Pratiquez la flottaison :** Apprenez à flotter sur votre dos et sur votre ventre. Cela vous aidera à vous sentir à l'aise dans l'eau.
* **Apprenez les coups de base :** Maîtrisez les mouvements de brasse, de crawl et de dos crawlé. Commencez par les pratiquer individuellement, puis combinez-les dans un nage continu.
* **Entraînez-vous régulièrement :** La pratique régulière est essentielle pour améliorer vos compétences. Visez au moins deux séances de natation par semaine.

**Niveau intermédiaire**

* **Travaillez sur votre vitesse et votre endurance :** Une fois que vous maîtr

# Prompt élaboré

In [5]:
def create_elaborate_prompt(user_request):
    return f"""
Vous êtes un expert conseil en loisirs spécialiste en fitness et en course à pied. Votre tâche consiste à créer une analyse détaillée, basée sur des données, et un plan de recommandations pour quelqu'un souhaitant s'initier à la course à pied en tant que loisir.

Voici la demande de l'utilisateur : {user_request}

Veuillez structurer votre réponse de la manière suivante :

1. Évaluation Initiale
- Analyser pourquoi la course à pied est un bon choix de loisir
- Liste des principaux avantages (physiques, mentaux, sociaux)
- Identifier les défis potentiels pour les débutants

2. Plan de Démarrage
- Équipement requis et coûts estimés
- Horaires recommandés pour les débutants
- Considérations de sécurité et prérequis

3. Chronogramme de Progression
- Décomposition semaine par semaine pour les 8 premières semaines
- Paliers mensuels pour les 6 premiers mois
- Objectifs à long terme et accomplissements

4. Analyse des Données
Veuillez inclure des tableaux de données dans votre analyse avec la structure suivante :

Coûts de l'Équipement :
| Article | Essentiel | Coût Moyen | Durée de Vie (Mois) |
|---------|-----------|------------|--------------------|

Progrès Hebdomadaire :
| Semaine | Minutes de Course | Minutes de Marche | Distance Totale (Km) | Séances Par Semaine |
|---------|-------------------|-------------------|---------------------|--------------------|

Paliers Mensuels :
| Mois | Objectif de Distance (Km) | Cadence Moyenne (Min/Km) | Longue Course (Km) |
|------|--------------------------|--------------------------|--------------------|

5. Suggestions de Visualisation
- Progrès de la distance hebdomadaire
- Améliorations de la cadence
- Zones de fréquence cardiaque
- Métriques de récupération

6. Communauté et Soutien
- Applications et outils recommandés
- Groupes locaux ou communautés
- Ressources en ligne

7. Analyse des Risques
Inclure une matrice d'évaluation des risques pour les défis courants et leurs solutions.

Veuillez fournir une réponse exhaustive, structurée de manière claire avec des recommandations spécifiques et opérationnelles.
"""

#### Extraire les Tableaux à partir du Texte

Explication :

La fonction extract_tables analyse les tableaux au format markdown à partir de la réponse texte de l'AI et les convertit en DataFrames Pandas.

In [6]:
def extract_tables(text):
    """Extraire les tableaux markdown du texte"""
    table_pattern = r"\|.*?\|[\r\n][-|:\s]+[\r\n](?:\|.*?\|[\r\n])+"
    tables = []
    
    for table_match in re.finditer(table_pattern, text, re.MULTILINE):
        table_text = table_match.group(0)
        
        lines = [line.strip() for line in table_text.split('\n') if line.strip()]
        headers = [col.strip() for col in lines[0].split('|')[1:-1]]
        data = []
        for line in lines[2:]:
            row = [cell.strip() for cell in line.split('|')[1:-1]]
            data.append(row)
        df = pd.DataFrame(data, columns=headers)
        tables.append(df)
        
    return tables

#### Extraire les Sections à partir du Texte

Explication

La fonction extract_sections identifie et extrait différentes sections de la réponse de l'AI en se basant sur les en-têtes ou les points numérotés.

In [7]:
def extract_sections(text):
    """Extraire les sections en se basant sur les points numérotés ou les en-têtes"""
    sections = {}
    current_section = None
    current_content = []
    
    for line in text.split('\n'):
        # Vérifier les en-têtes de sections (points numérotés ou en-têtes markdown)
        if re.match(r'^\d+\.|\#\#?|^\*\*', line.strip()):
            if current_section and current_content:  # Seulement enregistrer s'il y a du contenu
                content = '\n'.join(current_content).strip()
                if content:  # Seulement enregistrer si le contenu n'est pas vide
                    sections[current_section] = content
            current_section = line.strip('*# \n')
            current_content = []
        else:
            if current_section:
                current_content.append(line)
    
    # Ajouter la dernière section si elle a du contenu
    if current_section and current_content:
        content = '\n'.join(current_content).strip()
        if content:
            sections[current_section] = content
    
    return sections

### Test

In [8]:
from tabulate import tabulate
elaborate_prompt = create_elaborate_prompt(user_request)
elaborate_response = model.generate_content(elaborate_prompt).text

# Extraire et afficher les sections
sections = extract_sections(elaborate_response)
if sections:  # Afficher l'en-tête des sections seulement si des sections existent

    for section_title, content in sections.items():
        # Créer un expander seulement si le contenu existe et n'est pas seulement des espaces
        if content and not content.isspace():
            print(section_title)
            print("************************************")
            print(content)

# Extraire et afficher les tableaux
elaborate_tables = extract_tables(elaborate_response)
if elaborate_tables:
    print("Tableaux Trouvés dans la Réponse Elaborée")
    for i, table in enumerate(elaborate_tables):
        if not table.empty:  # Afficher seulement les tableaux non vides
            print(tabulate(table, headers='keys', tablefmt='psql'))
            




Pourquoi la course à pied est un bon choix de loisir :
************************************
* Accessible à tous les niveaux de forme physique
* Peu coûteux et pratique à pratiquer partout
* Améliore la santé physique et mentale
* Offre des opportunités de socialisation
Principaux avantages :
************************************
* **Physiques :** Amélioration de la santé cardiovasculaire, de la force musculaire, de la perte de poids et de la qualité du sommeil.
* **Mentaux :** Réduction du stress, amélioration de l'humeur, renforcement de la confiance en soi et amélioration des fonctions cognitives.
* **Sociaux :** Possibilité de rencontrer de nouvelles personnes, de participer à des courses et de faire partie d'une communauté.
Défis potentiels pour les débutants :
************************************
* Douleurs musculaires et articulaires
* Difficultés respiratoires
* Manque de motivation
* Blessures
Équipement requis et coûts estimés :
************************************
| Article | 